In [1]:
from functions import *
import resources

running


In [2]:
def loadProgram(file):
    fil = open(file,'r')
    lineno = 0
    address = 0
    while True:
        line = fil.readline()
        lineno += 1
        if (lineno == 1 or lineno == 2):
            continue
        if(line == '' or line == '\n'):
            break
        try:
            instruc = line
            instruc = instruc.strip('\n')
            instruc = instruc.split(',')[0]
            instruc = instruc.split(';')[0]
            instruc = instruc.strip(',')    #指令为字符串类型,十六进制
            instruc = instruc.strip(';')

            print(instruc, len(instruc))
            
            for i in range(0, len(instruc)//2):
                resources.mem[address] = instruc[2*i : 2*i+2]
                address += 1
        except:
            print(f'File {file} line {lineno} has error')
            pass
    fil.close

In [3]:
loadProgram("test_pipeline.coe")
print(resources.mem)
print(resources.reg)
print(resources.PC)

30F00100000000000000 20
6011 4
6022 4
6033 4
6044 4
00 2
['30', 'F0', '01', '00', '00', '00', '00', '00', '00', '00', '60', '11', '60', '22', '60', '33', '60', '44', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', '00', 

In [4]:
def check():
   #print(resources.stat)
   if(resources.stat == 1):
      return 1
   elif(resources.stat == 2):
      print("Error: Halt")
      return 0
   elif(resources.stat == 3):
      print("Error: ilegal address")
      return 0
   elif(resources.stat == 4):
      print("Error: ilegal instruction")
      return 0

instruction_no = 0


In [5]:
import threading

pre_signals = (0, 0, 0, 0, 0)
(IF_icode, IF_ifun, IF_rA, IF_rB, IF_valP, IF_valC) = (0, 0, 0, 0, 0, 0)
(ID_icode, ID_ifun, ID_valA, ID_valB, ID_valP, ID_valC, ID_rB, ID_rA) = (0,0,0,0,0,0,0,0)
(EX_valE, EX_valP, EX_Cnd,  EX_valC, EX_icode, EX_ifun, EX_valA, EX_rB, EX_rA) = (0,0,0,0,0,0,0,0,0)
(MEM_valP, MEM_Cnd, MEM_valC, MEM_icode, MEM_valM, MEM_rB, MEM_valE, MEM_ifun, MEM_rA) = (0,0,0,0,0,0,0,0,0)
# fetch阶段任务
def task_fetch():
   global IF_icode, IF_ifun, IF_rA, IF_rB, IF_valP, IF_valC, pre_signals
   (IF_icode, IF_ifun, IF_rA, IF_rB, IF_valP, IF_valC) = Fetch(pre_signals)
   # TODO
   # 目前未考虑任何分支或者跳转指令
   pre_signals = (IF_valP, 0, IF_valC, IF_icode, 0)
   # debug
   thread_id = threading.current_thread().name
   print(f"thread {thread_id} fetching")

def task_decode():
   global ID_icode, ID_ifun, ID_valA, ID_valB, ID_valP, ID_valC, ID_rB, ID_rA
   (ID_icode, ID_ifun, ID_valA, ID_valB, ID_valP, ID_valC, ID_rB, ID_rA)= Decode((IF_icode, IF_ifun, IF_rA, IF_rB, IF_valP, IF_valC))
   # debug
   thread_id = threading.current_thread().name
   print(f"thread {thread_id} decoding")

def task_execute():
   global EX_valE, EX_valP, EX_Cnd,  EX_valC, EX_icode, EX_ifun, EX_valA, EX_rB, EX_rA
   (EX_valE, EX_valP, EX_Cnd,  EX_valC, EX_icode, EX_ifun, EX_valA, EX_rB, EX_rA) = Execute((ID_icode, ID_ifun, ID_valA, ID_valB, ID_valP, ID_valC, ID_rB, ID_rA))
   # debug
   thread_id = threading.current_thread().name
   print(f"thread {thread_id} executing")

def task_memory():
   global MEM_valP, MEM_Cnd, MEM_valC, MEM_icode, MEM_valM, MEM_rB, MEM_valE, MEM_ifun, MEM_rA
   (MEM_valP, MEM_Cnd, MEM_valC, MEM_icode, MEM_valM, MEM_rB, MEM_valE, MEM_ifun, MEM_rA) = Memory((EX_valE, EX_valP, EX_Cnd,  EX_valC, EX_icode, EX_ifun, EX_valA, EX_rB, EX_rA))
   # debug
   thread_id = threading.current_thread().name
   print(f"thread {thread_id} memorying")

def task_writeback():
   WriteBack((MEM_valP, MEM_Cnd, MEM_valC, MEM_icode, MEM_valM, MEM_rB, MEM_valE, MEM_ifun, MEM_rA))
   # debug
   thread_id = threading.current_thread().name
   print(f"thread {thread_id} writingback")

def cycle():
   # 创建5个线程，每个线程完成一个阶段的工作
   fetch = threading.Thread(target=task_fetch)
   decode = threading.Thread(target = task_decode) 
   execute = threading.Thread(target=task_execute)
   memory = threading.Thread(target = task_memory)
   writeback = threading.Thread(target = task_writeback)

   fetch.start()
   decode.start()
   execute.start()
   memory.start()
   writeback.start()

   fetch.join()
   decode.join()
   execute.join()
   memory.join()
   writeback.join()

   if check() == False:
      return False

   return True

RuntimeError: cannot join thread before it is started

In [ ]:
while True:
    if not cycle():
        break

# print(resources.mem)
print(resources.reg)

thread Thread-5 fetching
thread Thread-6 decoding
thread Thread-7 executing
thread Thread-8 memorying
thread Thread-9 writingback
thread Thread-10 fetching
thread Thread-11 decoding
thread Thread-12 executing
thread Thread-13 memorying
thread Thread-14 writingback
thread Thread-15 fetching
thread Thread-16 decoding
thread Thread-17 executing
thread Thread-18 memorying
thread Thread-19 writingback
thread Thread-20 fetching
thread Thread-21 decoding
thread Thread-22 executing
thread Thread-23 memorying
thread Thread-24 writingback
thread Thread-25 fetching
thread Thread-26 decoding
thread Thread-27 executing
thread Thread-28 memorying
thread Thread-29 writingback
thread Thread-30 fetching
thread Thread-31 decoding
thread Thread-32 executing
thread Thread-33 memorying
thread Thread-34 writingback
Error: Halt
[1, 0, 0, 0, 1000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
